In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]


In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [8]:
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://127.0.0.1:5000")
for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/07/28 11:37:43 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.
2025/07/28 11:37:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/28 11:37:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/28 11:37:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/726948074554649449/runs/c32d80d9a68341f481d3b0b66651ae2c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/726948074554649449


2025/07/28 11:37:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/28 11:37:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/726948074554649449/runs/fa54a3b4a1a64e39bd925787b18ef9a3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/726948074554649449


2025/07/28 11:37:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/28 11:37:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/726948074554649449/runs/a069f109590e4bc4bca8f8cb2d49d469
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/726948074554649449


2025/07/28 11:37:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/726948074554649449/runs/d921bc70e7a148bda4874242744d14ae
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/726948074554649449


***Register the Model***

In [ ]:
# result = mlflow.register_model(
#     "runs:/d16076a3ec534311817565e6527539c0/sklearn-model", "sk-learn-random-forest-reg"
# ) # doc

model_name = 'XGB-Smote'
# run_id=input('Please type RunID')
# model_uri = f'runs:/{run_id}/model'

run_id = "d921bc70e7a148bda4874242744d14ae"
model_uri = "runs:/d921bc70e7a148bda4874242744d14ae/model"
with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/07/28 11:54:59 WARNING mlflow.tracking._model_registry.fluent: Run with id d921bc70e7a148bda4874242744d14ae has no artifacts at artifact path 'model', registering model based on models:/m-ab03b678bf20488b8fcf02782569f06d instead
2025/07/28 11:54:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 2


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/726948074554649449/runs/d921bc70e7a148bda4874242744d14ae
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/726948074554649449


Created version '2' of model 'XGB-Smote'.


***Load the Model***

In [20]:
# model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}") # doc

model_version = 2
model_uri = f"models:/{model_name}/{model_version}"
loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

***Transition the Model to Production***

In [21]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"
client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)


Successfully registered model 'anomaly-detection-prod'.
Copied version '2' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1753697448089, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1753697448089, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='d921bc70e7a148bda4874242744d14ae', run_link='', source='models:/XGB-Smote/2', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [22]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])